This notebook has the protocol used to create the raster layers from 
The GHM data we are going to use is a single raster with a continuous variable from 0 to 1. The resolution of this dataset is 1-km. Supplementary figure 11 from the GHM paper shows a summary (the median) of the GHm values accross 19 Anthromes). 
Anthrome data can be downloaded from [here](http://ecotope.org/anthromes/v2/data/) , [appendix 3](https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1111%2Fj.1466-8238.2010.00540.x&file=GEB_540_sm_AppS3.pdf) of the paper on Anthromes explains how the land has been classified.
Using this Anthrome Classification Algorithm, we will use the following human encroachment classification: 
```
Urban: 11: Urban/12: Mixed settlements
Rainfed agriculture: 21: Rice villages/23: Rainfed villages/32: Residential rainfed croplands/33: Populated croplands/34: Remote croplands
Irrigated agriculture: 22: Irrigated villages/31: Residential irrigated croplands
Rangeland: 24: Pastoral villages/41: Residential rangelands/42: Populated rangelands/43: Remote rangelands
Other: 51: Residential woodlands/52: Populated woodlands/53: Remote woodlands/54: Inhabited treeless and barren lands/61: Wild woodland
```

This would mean that we would have 4 that do not overlap:
- Urban
- Rainfed agriculture
- Irrigated agriculture
- Rangeland

In [1]:
import arcpy
from arcpy.sa import *
arcpy.env.workspace = "C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\Human Pressure"

Human pressure data was provided by MOL 
```
README
Global human modification layer
2019.11.27
THESE FILES ARE FOR INTERNAL DESIGN PURPOSES ONLY, AND SHOULD NOT BE
ADVERTISED, USED, OR SHARED OUTSIDE OF THE HALF-EARTH TEAM UNTIL FURTHER
NOTICE.
gHM_extended_maskGADM36_cea_20191127.grd: 1km raster layer of global human
modification (gHM), derived from Kennedy et al. (2019). The original
layer was transformed to an equal-area projection using bilinear
interpolation. It was then extended to all terrestrial surface area,
filling in missing areas with a value of 0 (e.g., Antarctica). The
was then masked to the GADM3.6 land layer*.
Kennedy, C. M., Oakleaf, J. R., Theobald, D. M., Baruch‐Mordo, S., &amp;
Kiesecker, J. (2019). Managing the middle: A shift in conservation priorities
based on the global human modification gradient. Global Change Biology,
25(3), 811-826. Data available at:
https://figshare.com/articles/Global_Human_Modification/7283087/1
GADM version 3.6 available at:
https://gadm.org/download_world.html
```

The anthrome data had to be resampled to match the projection of GHM layer and the cell size. 

In [16]:
arcpy.management.ProjectRaster(r"anthromes\anthro2_a2000.tif", 
                               r"Human_pressure_outputs.gdb\anthro2_a2000_cea", 
                               "PROJCS['WGS_1984_Cylindrical_Equal_Area',GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Cylindrical_Equal_Area'],PARAMETER['false_easting',0.0],PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],PARAMETER['standard_parallel_1',30.0],UNIT['Meter',1.0]]", "NEAREST", "7393.40090922411 7393.40090922411", None, None, "GEOGCS['GCS_WGS_1984',DATUM['D_WGS_1984',SPHEROID['WGS_1984',6378137.0,298.257223563]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]", "NO_VERTICAL")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\\\Human_pressure_outputs.gdb\\anthro2_a2000_cea'>

In [19]:
arcpy.management.Resample(r"Human_pressure_outputs.gdb\anthro2_a2000_cea", 
                          r"Human_pressure_outputs.gdb\anthro2_a2000_cea_Resample", 
                          "968 1030", "NEAREST")

<Result 'C:\\Users\\Vizuser\\Documents\\ArcGIS\\Projects\\Human Pressure\\Human_pressure_outputs.gdb\\anthro2_a2000_cea_Resample'>

We reclassify the data into 4 classes:
- 1 == Urban
- 2 == Rainfed
- 3 == Irrigated
- 4 == Rangelands

In [20]:
out_raster = arcpy.sa.Reclassify(r"Human_pressure_outputs.gdb\anthro2_a2000_cea_Resample", 
                                 "LABEL", 
                                 "'11: Urban' 1;'12: Mixed settlements' 1;'21: Rice villages' 2;'22: Irrigated villages' 3;'23: Rainfed villages' 2;'24: Pastoral villages' 4;'31: Residential irrigated croplands' 3;'32: Residential rainfed croplands' 2;'33: Populated croplands' 2;'34: Remote croplands' 2;'41: Residential rangelands' 4;'42: Populated rangelands' 4;'43: Remote rangelands' 4;'51: Residential woodlands' 0;'52: Populated woodlands' 0;'53: Remote woodlands' 0;'54: Inhabited treeless and barren lands' 0;'61: Wild woodlands' 0;'62: Wild treeless and barren lands' 0", "DATA");
out_raster.save("..\Human_pressure_outputs.gdb\Antro4classes")

And use `Antro4classes` to extract the values from gHM. 

In [21]:
outSetNull_urban = SetNull("Human_pressure_outputs.gdb\Antro4classes", "gHM_extended_maskGADM36_cea_20191127.tif", "VALUE <> 1")
outSetNull_urban.save("..\Human_pressure_outputs.gdb\gHM_Urban")

In [22]:
outSetNull_rainfed = SetNull("Human_pressure_outputs.gdb\Antro4classes", "gHM_extended_maskGADM36_cea_20191127.tif", "VALUE <> 2")
outSetNull_rainfed.save("..\Human_pressure_outputs.gdb\gHM_Rainfed")

In [23]:
outSetNull_irrigated = SetNull("Human_pressure_outputs.gdb\Antro4classes", "gHM_extended_maskGADM36_cea_20191127.tif", "VALUE <> 3")
outSetNull_irrigated.save("..\Human_pressure_outputs.gdb\gHM_Irrigated")

In [24]:
outSetNull_rangeland = SetNull("Human_pressure_outputs.gdb\Antro4classes", "gHM_extended_maskGADM36_cea_20191127.tif", "VALUE <> 4")
outSetNull_rangeland.save("..\Human_pressure_outputs.gdb\gHM_Rangeland")